# Actual Starting of Pipeline

# Imports and PIP installs

In [1]:
!pip install ultralytics

  Using cached ultralytics-8.3.156-py3-none-any.whl (1.0 MB)
  Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached pillow-11.2.1-cp310-cp310-win_amd64.whl (2.7 MB)
  Using cached pandas-2.3.0-cp310-cp310-win_amd64.whl (11.1 MB)


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.signal import find_peaks, savgol_filter
from scipy.ndimage import binary_closing
import secrets      # cryptographically‑secure RNG
import base64       # for compact ASCII/“number + letter” output
import uuid

# YOLO model prediction

In [3]:
# Input image
image_path = "kaggle/input/tests/a01-000x.png"

In [4]:
from ultralytics import YOLO

model = YOLO("kaggle/input/weights/last.pt")

WARNING Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\User\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
def seperate_handwritten_printed_using_yolo(image_path, folder_name):
    results = model([image_path])  # assuming `model` is already defined and loaded
    image = cv2.imread(image_path)
    base_name = os.path.basename(image_path)

    base_crop_folder = "cropped_outputs"
    base_graph_folder = "graph_outputs"

    for result in results:
        boxes = result.boxes
        cls = boxes.cls.cpu().numpy()
        xyxy = boxes.xyxy.cpu().numpy()

        class9_boxes = [box for i, box in enumerate(xyxy) if cls[i] == 9]

        def ensure_and_save_crop(box, root_folder):
            x1, y1, x2, y2 = map(int, box)
            cropped = image[y1:y2, x1:x2]
            subfolder = os.path.join(folder_name, root_folder)
            os.makedirs(subfolder, exist_ok=True)
            filename = f"{uuid.uuid4().hex}.jpg"
            print(cv2.imwrite(os.path.join(subfolder, filename), cropped))

        # Save all class 9 cropped regions
        for box in class9_boxes:
            ensure_and_save_crop(box, base_crop_folder)

        # Save image with detection graph
        graph = result.plot()
        graph_folder = os.path.join(folder_name, base_graph_folder)
        os.makedirs(graph_folder, exist_ok=True)
        print(cv2.imwrite(os.path.join(graph_folder, base_name), graph))

In [8]:
seperate_handwritten_printed_using_yolo(image_path, "kaggle/output/")


0: 192x640 2 Section-headers, 1 Text, 55.0ms
Speed: 1.4ms preprocess, 55.0ms inference, 1.2ms postprocess per image at shape (1, 3, 192, 640)
True
True


# Splitting lines from the detected image

In [9]:
# Update your base directories
base_input_dir = r"kaggle/output/cropped_outputs"
base_output_printed = r"kaggle/output/cropped_outputs_line"
base_graph_folder = r"kaggle/output/line_graphs"

In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.signal import find_peaks, savgol_filter
from scipy.ndimage import binary_closing

def auto_savgol_smooth(profile, polyorder=2, spacing_factor=None,
                       plot=True, plot_title="", save_path=None,
                       show_thresholds=False, high_thresh=None, low_thresh=None):
    
    peaks, _ = find_peaks(profile, distance=8)
    if len(peaks) < 2:
        raise ValueError("Not enough peaks detected to estimate line spacing.")

    diffs = np.diff(peaks)
    eps = 1e-9
    weights = 1.0 / (diffs + eps)
    avg_spacing = int(np.round(np.sum(weights * diffs) / np.sum(weights)))

    # Dynamically estimate spacing_factor if not provided
    if spacing_factor is None:
        spacing_factor = min(max(1.2, avg_spacing / 20), 2.0)

    window_length = int(spacing_factor * avg_spacing)
    if window_length % 2 == 0:
        window_length += 1
    window_length = max(window_length, polyorder + 4)
    window_length = min(window_length,
                        len(profile) - 1 if len(profile) % 2 else len(profile) - 2)

    smoothed = savgol_filter(profile, window_length=window_length, polyorder=polyorder)

    if plot:
        fig = plt.figure(figsize=(14, 5))
        plt.plot(profile, label="Original", color="orange", alpha=0.6)
        plt.plot(smoothed, label=f"Smoothed (window={window_length})", color="blue")
        plt.plot(peaks, profile[peaks], "rx", label="Detected Peaks")

        if show_thresholds:
            if high_thresh is not None:
                plt.axhline(y=high_thresh, color="red", linestyle="--", label=f"High Thresh = {high_thresh:.2f}")
            if low_thresh is not None:
                plt.axhline(y=low_thresh, color="green", linestyle="--", label=f"Low Thresh = {low_thresh:.2f}")

        plt.title(plot_title or "Savitzky-Golay smoothing")
        plt.xlabel("Row Index")
        plt.ylabel("Sum of Pixel Intensities")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        if save_path is not None:
            fig.savefig(save_path)
        plt.close(fig)

    return smoothed, spacing_factor



def calculate_projection_profile_and_crop_lines_with_lines(image_path, folder_name):
    base_name = os.path.basename(image_path)
    image_name_no_ext = os.path.splitext(base_name)[0]

    subfolder_graph = os.path.join(base_graph_folder, folder_name)
    os.makedirs(subfolder_graph, exist_ok=True)
    output_path = os.path.join(subfolder_graph, f"{base_name}")

    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        return

    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    horizontal_projection = np.sum(binary_image, axis=1)

    smoothed, spacing_factor = auto_savgol_smooth(
        horizontal_projection,
        save_path=output_path,
        plot=True,
        show_thresholds=True
    )

    # === Dynamic Thresholds ===
    Q1 = np.percentile(smoothed, 25)
    Q3 = np.percentile(smoothed, 75)
    IQR = Q3 - Q1
    mean_val = np.mean(smoothed)
    min_val = np.min(smoothed)
    max_val = np.max(smoothed)

    iqr_low = Q1 + 0.2 * IQR
    iqr_high = iqr_low + 0.25 * IQR
    mean_low = mean_val * 0.25
    mean_high = mean_val * 0.6
    scaled_low = min_val + 0.1 * (max_val - min_val)
    scaled_high = min_val + 0.35 * (max_val - min_val)

    low_thresh = np.median([iqr_low, mean_low, scaled_low])
    high_thresh = np.median([iqr_high, mean_high, scaled_high])

    # Re-plot with thresholds
    smoothed, _ = auto_savgol_smooth(
        horizontal_projection,
        spacing_factor=spacing_factor,
        save_path=output_path,
        plot=True,
        show_thresholds=True,
        high_thresh=high_thresh,
        low_thresh=low_thresh
    )

    # === Line Detection with Relaxed High Threshold at Bottom ===
    line_ranges = []
    is_in_line = False
    relaxed_zone = int(0.8 * len(smoothed))

    for row, value in enumerate(smoothed):
        current_high = high_thresh
        if row > relaxed_zone:
            current_high = high_thresh * 0.7  # relax threshold in bottom zone

        if value > current_high and not is_in_line:
            start_row = row
            is_in_line = True
        elif value < low_thresh and is_in_line:
            end_row = row
            line_ranges.append((start_row, end_row))
            is_in_line = False

    if is_in_line:
        line_ranges.append((start_row, len(smoothed)))

    # === Fallback: Recover Missed Final Line ===
    last_line_margin = int(len(smoothed) * 0.17)
    end_threshold = len(smoothed) - last_line_margin
    last_part_vals = smoothed[-last_line_margin:]

    if all(end < end_threshold for _, end in line_ranges):
        if np.max(last_part_vals) > low_thresh:
            fallback_start = end_threshold
            line_ranges.append((fallback_start, len(smoothed)))

    # === Refine Borders ===
    if line_ranges:
        line_ranges[0] = (max(0, line_ranges[0][0] - 5), line_ranges[0][1])
        line_ranges[-1] = (line_ranges[-1][0], min(image.shape[0], line_ranges[-1][1] + 5))

    for i in range(1, len(line_ranges)):
        temp = (line_ranges[i - 1][1] + line_ranges[i][0]) // 2
        line_ranges[i - 1] = (line_ranges[i - 1][0], temp)
        line_ranges[i] = (temp, line_ranges[i][1])

    line_ranges = sorted(line_ranges, key=lambda x: x[0])

    # === Save Cropped Lines ===
    subfolder_output = os.path.join(base_output_printed, folder_name, image_name_no_ext)
    os.makedirs(subfolder_output, exist_ok=True)

    for idx, (start, end) in enumerate(line_ranges, 1):
        cropped_line = image[start:end, :]
        save_path = os.path.join(subfolder_output, f"{idx}.png")
        print(f'save_path = {save_path}')
        cv2.imwrite(save_path, cropped_line)

In [11]:
# === Batch processor ===
def process_all_images():
    for root, dirs, files in os.walk(base_input_dir):
        image_files = sorted(
            [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
        )
        for file in image_files:
            image_path = os.path.join(root, file)
            folder_name = os.path.basename(root)
            try:
                calculate_projection_profile_and_crop_lines_with_lines(image_path, folder_name)
            except Exception as e:
                print(f"Error processing {image_path}: {e}")

In [12]:
process_all_images()

save_path = kaggle/output/cropped_outputs_line\cropped_outputs\ccb78bea7c4848f0b41dc297f7fbda07\1.png
save_path = kaggle/output/cropped_outputs_line\cropped_outputs\ccb78bea7c4848f0b41dc297f7fbda07\2.png
save_path = kaggle/output/cropped_outputs_line\cropped_outputs\ccb78bea7c4848f0b41dc297f7fbda07\3.png
save_path = kaggle/output/cropped_outputs_line\cropped_outputs\ccb78bea7c4848f0b41dc297f7fbda07\4.png


# Predicting whether Image is handwritten or printed

In [13]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import torch.nn as nn

# Constants
IMG_SIZE = 128
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the same model class
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.fc = nn.Linear(128, 2)

    def forward(self, x):
        x = self.net(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# Create model and load weights
model = SimpleCNN().to(DEVICE)
model.load_state_dict(torch.load("kaggle/input/weights/final_model_weights_HvP.pth", map_location=DEVICE))
model.eval()

# Test transforms
test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [14]:
from PIL import Image
import torch

# --- Load and preprocess the image ---
def predict_HvP(image_path):
    image = Image.open(image_path).convert("RGB")
    image_tensor = test_transform(image).unsqueeze(0).to(DEVICE)  # Add batch dimension
    
    # --- Make prediction ---
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        predicted_class = output.argmax(1).item()
    
    # --- Map class index to class name ---
    class_names = ["handwritten", "printed"]  # Get class names from dataset
    print(f"Predicted class label: {class_names[predicted_class]}")
    return predicted_class

# Predicting text from image using TrOCR

In [17]:
from PIL import Image
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import os

# Load models and processors only once
hand_written_model_id = "microsoft/trocr-large-handwritten"
printed_model_id = "microsoft/trocr-base-printed"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on {device}")

# Load both models and processors once
printed_processor = TrOCRProcessor.from_pretrained(printed_model_id)
printed_model = VisionEncoderDecoderModel.from_pretrained(printed_model_id).to(device)

handwritten_processor = TrOCRProcessor.from_pretrained(hand_written_model_id)
handwritten_model = VisionEncoderDecoderModel.from_pretrained(hand_written_model_id).to(device)

# OCR runner
def run_trOCR(model, processor, image_path):
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
    generated_ids = model.generate(pixel_values, max_new_tokens=1000)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(f"{os.path.basename(image_path)} -> {generated_text}")
    return generated_text

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on cpu


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--microsoft--trocr-base-printed. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Using a slow image processor as `use_fast` is unset and a slow processor was saved with

# Final Prediction for the texts

In [18]:
# Path to input images
folder_dir = "kaggle/output/cropped_outputs_line/cropped_outputs/"
input_dir = os.path.join(folder_dir,os.listdir(folder_dir)[0])

In [19]:
# Collect image file paths
file_paths = [
    os.path.join(input_dir, filename)
    for filename in sorted(os.listdir(input_dir))  # sort if needed
    if os.path.isfile(os.path.join(input_dir, filename))
]

# Loop through and run OCR
for path in file_paths:
    if predict_HvP(path) == 1:
        run_trOCR(printed_model, printed_processor, path)
    else:
        run_trOCR(handwritten_model, handwritten_processor, path)

Predicted class label: printed
1.png -> A MOVE TO STOP MR. GAITSKELL FROM DOMINATING ANY MAKE LABOR LIFE IS TO
Predicted class label: printed
2.png -> BE MADE AT A MEETING OF LABOUR OM PS TOMORROW. MR. MIDHAEL FOOT HAS PUT DOWN
Predicted class label: printed
3.png -> A RESOLUTION ON THE SUBJECT AND BE IS TO BE BACKED BY MR. WILL GRIFITHS, OM P FOR
Predicted class label: printed
4.png -> MANCHASTER EXCHANGE.COME AGAIN PERMAS AND BEAUTIONS
